# Check accuracy of kNN imputation

Note: kNN imputation is no longer used in the cost estimation pipeline.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.impute import KNNImputer
from tqdm.notebook import tqdm

from imputation import *
from plotting import *

In [3]:
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='sklearn.neighbors')

## Load the known data

In [4]:
pcd_df = pd.read_csv('data/All ML Systems - full view.csv')
pcd_df.head()

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Open-source,Link,Citations,Reference,...,Organization (from Organization),Base model,Finetune compute (FLOP),Finetune compute notes,Authors by country,Hardware quantity,Hardware utilization,Training cost trends,Training cloud compute vendor,Training data center
0,Cohere Command,Language,NaN,NaN,NaN,NaN,NaN,https://cohere.com/models/command,NaN,"World-class AI, at your command",...,Cohere,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Theseus,Other,Maze solving,Claude Shannon,Historical significance,NaN,NaN,https://www.technologyreview.com/2018/12/19/13...,0.0,Mighty Mouse,...,Bell Laboratories,NaN,NaN,NaN,Theseus,NaN,NaN,NaN,NaN,NaN
2,SNARC,Other,Maze solving,Marvin Minsky,Historical significance,NaN,NaN,https://en.wikipedia.org/wiki/Stochastic_neura...,33.0,A Neural-Analogue Calculator Based upon a Prob...,...,Harvard University,NaN,NaN,NaN,SNARC,NaN,NaN,NaN,NaN,NaN
3,Genetic algorithm,NaN,NaN,NA Barricelli,Historical significance,Possibly first computer simulation of a geneti...,NaN,https://link.springer.com/article/10.1007/BF01...,266.0,Numerical testing of evolution theories,...,Institute for Advanced Study,NaN,NaN,NaN,Genetic algorithm,NaN,NaN,NaN,NaN,NaN
4,Sequence-based pattern recognition,Vision,Character recognition,O. G. Selfridge,Historical significance,NaN,NaN,https://dl.acm.org/doi/10.1145/1455292.1455310,290.0,Pattern recognition and modern computers,...,Massachusetts Institute of Technology (MIT),NaN,NaN,NaN,Sequence-based pattern recognition,NaN,NaN,NaN,NaN,NaN


In [5]:
# Publication date in float format
pcd_df.dropna(subset=['Publication date'], inplace=True)
pcd_df['Publication date'] = pd.to_datetime(pcd_df['Publication date'])
pcd_df['Publication date'] = pcd_df['Publication date'].dt.year + (pcd_df['Publication date'].dt.month-1)/12 + (pcd_df['Publication date'].dt.day-1)/365

In [6]:
# Manually copied from "Training cost trends" Airtable
frontier_systems = [
    "PaLM 2",
    "GPT-4",
    "Minerva (540B)",
    "Megatron-Turing NLG 530B",
    "GPT-3 175B (davinci)",
    "Meena",
    "AlphaStar",
    "AlphaGo Zero",
    "AlphaGo Master",
    "GNMT",
    "Claude 2",
    "PaLM (540B)",
    "ERNIE 3.0 Titan",
    "Gopher (280B)",
    "OpenAI Five",
    "T5-11B",
    "Megatron-BERT",
    "ResNeXt-101 32x48d",
    "AlphaZero",
    "Falcon 180B",
    "GPT-3.5 (text-davinci-003)",
    "Chinchilla",
    "Yuan 1.0",
    "Turing-NLG",
    "BigGAN-deep 512x512",
    "NASv3 (CIFAR-10)",
    "AlphaGo Lee",
    "AlphaGo Fan",
    "OPT-175B",
    "AlphaCode",
    "GLaM",
    "OpenAI Five Rerun",
    "T5-3B",
    "Megatron-LM (8.3B)",
    "FTW",
    "AmoebaNet-A (F=448)",
    "OpenAI TI7 DOTA 1v1",
    "JFT",
    "Llama 2-70B",
    "LLaMA-65B",
    "LaMDA",
    "ALIGN",
    "GShard (dense)",
    "RoBERTa Large",
    "IMPALA",
]

In [7]:
frontier_pcd_df = pcd_df[pcd_df['System'].isin(frontier_systems)]
frontier_pcd_df.head()

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Open-source,Link,Citations,Reference,...,Organization (from Organization),Base model,Finetune compute (FLOP),Finetune compute notes,Authors by country,Hardware quantity,Hardware utilization,Training cost trends,Training cloud compute vendor,Training data center
265,AlphaGo Fan,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",SOTA improvement,NaN,NaN,https://www.nature.com/articles/nature24270.ep...,14389.0,Mastering the game of Go with deep neural netw...,...,Google DeepMind,NaN,NaN,NaN,AlphaGo Fan,NaN,NaN,AlphaGo Fan,NaN,NaN
275,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14389.0,Mastering the game of Go with deep neural netw...,...,DeepMind,NaN,NaN,NaN,AlphaGo Lee,NaN,NaN,AlphaGo Lee,NaN,NaN
306,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,NaN,https://arxiv.org/abs/1609.08144,5948.0,Google's Neural Machine Translation System: Br...,...,Google,NaN,NaN,NaN,GNMT,96.0,NaN,GNMT,NaN,NaN
317,NASv3 (CIFAR-10),Vision,NaN,"Barret Zoph, Quoc V. Le",Highly cited,NaN,NaN,https://arxiv.org/abs/1611.01578,4569.0,Neural Architecture Search with Reinforcement ...,...,Google Brain,NaN,NaN,NaN,NASv3 (CIFAR-10),800.0,NaN,NASv3 (CIFAR-10),NaN,NaN
337,AlphaGo Master,Games,Go,"D Silver, J Schrittwieser, K Simonyan, I Anton...",Highly cited,NaN,NaN,https://www.researchgate.net/publication/32047...,7831.0,Mastering the game of Go without human knowledge,...,DeepMind,NaN,NaN,NaN,AlphaGo Master,NaN,NaN,AlphaGo Master,NaN,NaN


In [8]:
pcd_hardware_model_colname = 'Name of the hardware (from Training hardware)'

## Prepare for imputation

In [9]:
frontier_pcd_df = frontier_pcd_df.set_index('System')

In [10]:
# drop unneeded columns from frontier_pcd_df
irrelevant_columns = ['Notability criteria', 'Notability criteria notes', 'Link', 'Citations', 'Parameters notes',
                      'Training compute notes', 'Training dataset notes', 'Dataset size notes',
                      'Inference compute notes', 'Approach', 'Confidence', 'Last modified', 'Created By', 'Benchmark data',
                      'Exclude', 'Authors by country', 'Training cost trends', 'Abstract', 'Compute cost notes',
                      'Training time notes', 'Authors', 'Name of the hardware (from Training hardware)',
                      'Training compute cost (2020 USD)', 'Organization categorization',
                      'Training dataset', 'Inference compute (FLOP)', 'Compute sponsor categorization',
                      'Finetune compute notes']
frontier_pcd_df = frontier_pcd_df.drop(columns=irrelevant_columns)

In [11]:
# fill column 'Training cloud compute vendor' using org_to_cloud_vendor dictionary
org_to_cloud_vendor = {
    'Google': 'Google Cloud',
    'DeepMind': 'Google Cloud',
    'Google DeepMind': 'Google Cloud',
    'Google Brain': 'Google Cloud',
    'Microsoft': 'Microsoft Azure',
    'OpenAI': 'Microsoft Azure',
}
frontier_pcd_df['Training cloud compute vendor'] = frontier_pcd_df['Organization (from Organization)'].map(org_to_cloud_vendor)
frontier_pcd_df['Training cloud compute vendor'] = frontier_pcd_df['Training cloud compute vendor'].fillna('Amazon Web Services')

In [12]:
# convert large number columns to logarithmic
frontier_pcd_df['log_params'] = np.log10(frontier_pcd_df['Parameters'])
frontier_pcd_df['log_compute'] = np.log10(frontier_pcd_df['Training compute (FLOP)'])
frontier_pcd_df['log_dataset'] = np.log10(frontier_pcd_df['Training dataset size (datapoints)'])
# drop raw columns
raw_columns = ['Parameters', 'Training compute (FLOP)', 'Training dataset size (datapoints)']
frontier_pcd_df.drop(columns=raw_columns, inplace=True)

In [13]:
frontier_pcd_df.columns

Index(['Domain', 'Task', 'Open-source', 'Reference', 'Publication date',
       'Organization', 'Epochs', 'Training time (hours)', 'Training hardware',
       'Country (from Organization)', 'Organization (from Organization)',
       'Base model', 'Finetune compute (FLOP)', 'Hardware quantity',
       'Hardware utilization', 'Training cloud compute vendor',
       'Training data center', 'log_params', 'log_compute', 'log_dataset'],
      dtype='object')

In [14]:
num_known_hardware_models = frontier_pcd_df['Training hardware'].notna().sum()

In [15]:
print(
    "Total rows: ", frontier_pcd_df.shape[0], "\n",
    "Known hardware models: ", num_known_hardware_models, "\n",
    "Known hardware utilization: ", frontier_pcd_df['Hardware utilization'].notna().sum(), "\n",
    "Known hardware quantity: ", frontier_pcd_df['Hardware quantity'].notna().sum(), "\n",
    "Known training time: ", frontier_pcd_df['Training time (hours)'].notna().sum(),
    sep="",
)

Total rows: 45
Known hardware models: 35
Known hardware utilization: 14
Known hardware quantity: 33
Known training time: 27


Next, for different values of k, perform imputation and calculate the MSE for imputed quantitative data and accuracy for categorical data.

## Run imputation many times and evaluate

In [16]:
num_trials = 1000
num_held_out = 5
k_values = [1, 2, 4, 9, 16, 25, num_known_hardware_models - num_held_out]
results = {k: [] for k in k_values}

In [18]:
frontier_pcd_df.columns

Index(['Domain', 'Task', 'Open-source', 'Reference', 'Publication date',
       'Organization', 'Epochs', 'Training time (hours)', 'Training hardware',
       'Country (from Organization)', 'Organization (from Organization)',
       'Base model', 'Finetune compute (FLOP)', 'Hardware quantity',
       'Hardware utilization', 'Training cloud compute vendor',
       'Training data center', 'log_params', 'log_compute', 'log_dataset'],
      dtype='object')

In [19]:
for i in tqdm(range(num_trials)):
    for k in k_values:
        # TODO: put some of this block in a function to avoid code duplication between here and cost notebook

        # Drop 5 random rows with known hardware model
        train_df, holdout_models = drop_random_values(frontier_pcd_df, 'Training hardware', num_held_out)

        # Robi: TODO I think we should download the latest CSV now that "name of the hardware" redundant column is removed
        # Then adjust the names of the cols we're dropping so it reflects the current structure of the table

        # TODO The best k value might be different for utilization and hardware
        # So might be best to have different cells tuning k for each

        # TODO: should we be using imputed numerical values as features for imputing training hardware?
        # It could be garbage in, garbage out
        imputed_pcd_df = knn_impute_numerical_pcd_data(train_df, num_neighbors=k)

        # Impute training hardware separately, because it is a categorical variable
        # There could be a better solution to this, but it seems complicated no matter what - see https://stackoverflow.com/questions/64900801/implementing-knn-imputation-on-categorical-variables-in-an-sklearn-pipeline
        imputed_pcd_df = knn_impute_categorical_column(
            imputed_pcd_df,
            num_neighbors=k,
            target_col='Training hardware'
        )

        # Restore the System column
        imputed_pcd_df['System'] = train_df.index

        # set the System column as the index
        imputed_pcd_df = imputed_pcd_df.set_index('System')

        # insert imputed values into train_df
        train_df['Training hardware'] = imputed_pcd_df['Training hardware']
        train_df['Hardware quantity'] = imputed_pcd_df['Hardware quantity']
        train_df['Hardware utilization'] = imputed_pcd_df['Hardware utilization']
        train_df['Training time (hours)'] = imputed_pcd_df['Training time (hours)']

        # calculate accuracy
        true_hardware = holdout_models['Training hardware']
        imputed_hardware = train_df.loc[holdout_models.index, 'Training hardware']
        accuracy = np.sum(true_hardware == imputed_hardware)/num_held_out
        results[k].append(accuracy)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [20]:
results = {k: round(np.mean(v), 4) for k, v in results.items()}
print(results)

{1: 0.1214, 2: 0.1362, 4: 0.1608, 9: 0.2462, 16: 0.247, 25: 0.2612, 30: 0.251}
